In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def firsts(list):
    answer=[]
    for pair in list:
        answer=answer+[pair[0]]
    return answer

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jLeoPowerConstantTerm(m,power,multiplier):
    trun=jLeo(power,m,multiplier)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def last_element(listo):return listo[len(listo)-1]
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

print("okay")

okay


In [ ]:
#output snipped; I interrupted this after k = 94.
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [1..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-nszfunction(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
#below was originally done in a separate cell
#import pickle
#wfile = open('/Users/barrybrent/26oct22no1.txt','wb')
#t = pickle.dumps(str(polydata))
#wfile.write(t)
#wfile.close()
---------------------------------------------------------------------------------------------------
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [95..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst=QQ(jpowerConstantTerm(m,power))
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-broadhurst(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
import pickle
wfile = open('/Users/barrybrent/26oct22no2.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
print("tests:")
print(tests)

In [2]:
a=[1,2,3];b=[4,5,6];c=a+b;print(c)

[1, 2, 3, 4, 5, 6]


In [6]:
import pickle
rsfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

rtfile = open('/Users/barrybrent/26oct22no2.txt','rb')
rt=pickle.load(rtfile)
rtfile.close()
t=stripQuotationMarks(rt)

u=s+t

wufile = open('/Users/barrybrent/5dec22no1.txt','wb')
u = pickle.dumps(str(u))
wufile.write(u)

1968604

In [8]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(len(s))

100


In [2]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=2
dspk=1
data=[]
for m in [3..2003]:
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            data=data+[ratio]
            #print((p,k,dspk,m,ratio))
ud=uniteList(data)
ud=sortAndSmash(ud)
print("prime: ",p," ratios:");print(ud)
print("==========================================================================================")
for rd in ud:
    print("prime: ",p," ratio:",rd," dspk: ",dspk)
    ld=len(data)  
    positions=[]
    for j in [0..ld-1]:
        if data[j]==rd:positions=positions+[j]
    print("first position:",positions[0])
    d=difference_table(positions)
    print("difference_table(positions):");print(d);
    print("---------------------------------------------------------------------------------------")

prime:  2  ratios:
[3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 19, 21, 23, 25, 27, 29, 31, 35, 39, 43, 47, 51, 57, 59, 67, 75, 83, 91, 99, 113, 115, 131, 147, 163, 179, 195, 225, 227, 259, 291, 323, 355, 387, 449, 451, 515, 579, 643, 707, 771]
prime:  2  ratio: 3  dspk:  1
first position: 0
difference_table(positions):
[1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 1, 

first position: 37
difference_table(positions):
[112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 55, 57, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 55, 57, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 55, 57, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 55, 57, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112, 112]
---------------------------------------------------------------------------------------
prime:  2  ratio: 25  dspk:  1
first position: 3564
difference_table(positions):
[7168]
---------------------------------------------------------------------------------------
prime:  2  ratio: 27  dspk:  1
first position: 93
diffe

In [ ]:
# I only have data for k <=99 or 100, so I can't go beyond p^4 for p = 3.
# Several OEIS sequences begin with 0,2,8,26,80.

In [94]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
mp=0
data=[]
p=2
tests=[]
tupls=[]
dk=1
for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dk: 
            poly=D(s[h][1])
            for m in [3..4100]:
                if ZZ(mod(m,p))==mp: # mp = 0 => m is even
                    #dm=ZZ(m/p).digits(p)
                    dm=digitsum(p,m)
                    a=order(p,m)
                    mp2=mod(m,p^2) # p = 2 => mp2 = mod(m,4); m is even, so mp2 = 0 or 2.
                    if mp2==0:
                            polym=evaluate(poly,m)
                            o=order(p,polym)
                            test=o-((a+2)*k+3)
                            tupl=(k,m,test)
                            tests=tests+[test]
                            print(tupl)
                            tupls=tupls+[tupl]
ut=uniteList(tests);print();print("tests:",ut)

(1, 4, 0)
(1, 8, 0)
(1, 12, 0)
(1, 16, 0)
(1, 20, 0)
(1, 24, 0)
(1, 28, 0)
(1, 32, 0)
(1, 36, 0)
(1, 40, 0)
(1, 44, 0)
(1, 48, 0)
(1, 52, 0)
(1, 56, 0)
(1, 60, 0)
(1, 64, 0)
(1, 68, 0)
(1, 72, 0)
(1, 76, 0)
(1, 80, 0)
(1, 84, 0)
(1, 88, 0)
(1, 92, 0)
(1, 96, 0)
(1, 100, 0)
(1, 104, 0)
(1, 108, 0)
(1, 112, 0)
(1, 116, 0)
(1, 120, 0)
(1, 124, 0)
(1, 128, 0)
(1, 132, 0)
(1, 136, 0)
(1, 140, 0)
(1, 144, 0)
(1, 148, 0)
(1, 152, 0)
(1, 156, 0)
(1, 160, 0)
(1, 164, 0)
(1, 168, 0)
(1, 172, 0)
(1, 176, 0)
(1, 180, 0)
(1, 184, 0)
(1, 188, 0)
(1, 192, 0)
(1, 196, 0)
(1, 200, 0)
(1, 204, 0)
(1, 208, 0)
(1, 212, 0)
(1, 216, 0)
(1, 220, 0)
(1, 224, 0)
(1, 228, 0)
(1, 232, 0)
(1, 236, 0)
(1, 240, 0)
(1, 244, 0)
(1, 248, 0)
(1, 252, 0)
(1, 256, 0)
(1, 260, 0)
(1, 264, 0)
(1, 268, 0)
(1, 272, 0)
(1, 276, 0)
(1, 280, 0)
(1, 284, 0)
(1, 288, 0)
(1, 292, 0)
(1, 296, 0)
(1, 300, 0)
(1, 304, 0)
(1, 308, 0)
(1, 312, 0)
(1, 316, 0)
(1, 320, 0)
(1, 324, 0)
(1, 328, 0)
(1, 332, 0)
(1, 336, 0)
(1, 340, 0)
(1, 34

(1, 3348, 0)
(1, 3352, 0)
(1, 3356, 0)
(1, 3360, 0)
(1, 3364, 0)
(1, 3368, 0)
(1, 3372, 0)
(1, 3376, 0)
(1, 3380, 0)
(1, 3384, 0)
(1, 3388, 0)
(1, 3392, 0)
(1, 3396, 0)
(1, 3400, 0)
(1, 3404, 0)
(1, 3408, 0)
(1, 3412, 0)
(1, 3416, 0)
(1, 3420, 0)
(1, 3424, 0)
(1, 3428, 0)
(1, 3432, 0)
(1, 3436, 0)
(1, 3440, 0)
(1, 3444, 0)
(1, 3448, 0)
(1, 3452, 0)
(1, 3456, 0)
(1, 3460, 0)
(1, 3464, 0)
(1, 3468, 0)
(1, 3472, 0)
(1, 3476, 0)
(1, 3480, 0)
(1, 3484, 0)
(1, 3488, 0)
(1, 3492, 0)
(1, 3496, 0)
(1, 3500, 0)
(1, 3504, 0)
(1, 3508, 0)
(1, 3512, 0)
(1, 3516, 0)
(1, 3520, 0)
(1, 3524, 0)
(1, 3528, 0)
(1, 3532, 0)
(1, 3536, 0)
(1, 3540, 0)
(1, 3544, 0)
(1, 3548, 0)
(1, 3552, 0)
(1, 3556, 0)
(1, 3560, 0)
(1, 3564, 0)
(1, 3568, 0)
(1, 3572, 0)
(1, 3576, 0)
(1, 3580, 0)
(1, 3584, 0)
(1, 3588, 0)
(1, 3592, 0)
(1, 3596, 0)
(1, 3600, 0)
(1, 3604, 0)
(1, 3608, 0)
(1, 3612, 0)
(1, 3616, 0)
(1, 3620, 0)
(1, 3624, 0)
(1, 3628, 0)
(1, 3632, 0)
(1, 3636, 0)
(1, 3640, 0)
(1, 3644, 0)
(1, 3648, 0)
(1, 3652, 0)

(2, 3428, 0)
(2, 3432, 0)
(2, 3436, 0)
(2, 3440, 0)
(2, 3444, 0)
(2, 3448, 0)
(2, 3452, 0)
(2, 3456, 0)
(2, 3460, 0)
(2, 3464, 0)
(2, 3468, 0)
(2, 3472, 0)
(2, 3476, 0)
(2, 3480, 0)
(2, 3484, 0)
(2, 3488, 0)
(2, 3492, 0)
(2, 3496, 0)
(2, 3500, 0)
(2, 3504, 0)
(2, 3508, 0)
(2, 3512, 0)
(2, 3516, 0)
(2, 3520, 0)
(2, 3524, 0)
(2, 3528, 0)
(2, 3532, 0)
(2, 3536, 0)
(2, 3540, 0)
(2, 3544, 0)
(2, 3548, 0)
(2, 3552, 0)
(2, 3556, 0)
(2, 3560, 0)
(2, 3564, 0)
(2, 3568, 0)
(2, 3572, 0)
(2, 3576, 0)
(2, 3580, 0)
(2, 3584, 0)
(2, 3588, 0)
(2, 3592, 0)
(2, 3596, 0)
(2, 3600, 0)
(2, 3604, 0)
(2, 3608, 0)
(2, 3612, 0)
(2, 3616, 0)
(2, 3620, 0)
(2, 3624, 0)
(2, 3628, 0)
(2, 3632, 0)
(2, 3636, 0)
(2, 3640, 0)
(2, 3644, 0)
(2, 3648, 0)
(2, 3652, 0)
(2, 3656, 0)
(2, 3660, 0)
(2, 3664, 0)
(2, 3668, 0)
(2, 3672, 0)
(2, 3676, 0)
(2, 3680, 0)
(2, 3684, 0)
(2, 3688, 0)
(2, 3692, 0)
(2, 3696, 0)
(2, 3700, 0)
(2, 3704, 0)
(2, 3708, 0)
(2, 3712, 0)
(2, 3716, 0)
(2, 3720, 0)
(2, 3724, 0)
(2, 3728, 0)
(2, 3732, 0)

(4, 2452, 0)
(4, 2456, 0)
(4, 2460, 0)
(4, 2464, 0)
(4, 2468, 0)
(4, 2472, 0)
(4, 2476, 0)
(4, 2480, 0)
(4, 2484, 0)
(4, 2488, 0)
(4, 2492, 0)
(4, 2496, 0)
(4, 2500, 0)
(4, 2504, 0)
(4, 2508, 0)
(4, 2512, 0)
(4, 2516, 0)
(4, 2520, 0)
(4, 2524, 0)
(4, 2528, 0)
(4, 2532, 0)
(4, 2536, 0)
(4, 2540, 0)
(4, 2544, 0)
(4, 2548, 0)
(4, 2552, 0)
(4, 2556, 0)
(4, 2560, 0)
(4, 2564, 0)
(4, 2568, 0)
(4, 2572, 0)
(4, 2576, 0)
(4, 2580, 0)
(4, 2584, 0)
(4, 2588, 0)
(4, 2592, 0)
(4, 2596, 0)
(4, 2600, 0)
(4, 2604, 0)
(4, 2608, 0)
(4, 2612, 0)
(4, 2616, 0)
(4, 2620, 0)
(4, 2624, 0)
(4, 2628, 0)
(4, 2632, 0)
(4, 2636, 0)
(4, 2640, 0)
(4, 2644, 0)
(4, 2648, 0)
(4, 2652, 0)
(4, 2656, 0)
(4, 2660, 0)
(4, 2664, 0)
(4, 2668, 0)
(4, 2672, 0)
(4, 2676, 0)
(4, 2680, 0)
(4, 2684, 0)
(4, 2688, 0)
(4, 2692, 0)
(4, 2696, 0)
(4, 2700, 0)
(4, 2704, 0)
(4, 2708, 0)
(4, 2712, 0)
(4, 2716, 0)
(4, 2720, 0)
(4, 2724, 0)
(4, 2728, 0)
(4, 2732, 0)
(4, 2736, 0)
(4, 2740, 0)
(4, 2744, 0)
(4, 2748, 0)
(4, 2752, 0)
(4, 2756, 0)

(8, 1960, 0)
(8, 1964, 0)
(8, 1968, 0)
(8, 1972, 0)
(8, 1976, 0)
(8, 1980, 0)
(8, 1984, 0)
(8, 1988, 0)
(8, 1992, 0)
(8, 1996, 0)
(8, 2000, 0)
(8, 2004, 0)
(8, 2008, 0)
(8, 2012, 0)
(8, 2016, 0)
(8, 2020, 0)
(8, 2024, 0)
(8, 2028, 0)
(8, 2032, 0)
(8, 2036, 0)
(8, 2040, 0)
(8, 2044, 0)
(8, 2048, 0)
(8, 2052, 0)
(8, 2056, 0)
(8, 2060, 0)
(8, 2064, 0)
(8, 2068, 0)
(8, 2072, 0)
(8, 2076, 0)
(8, 2080, 0)
(8, 2084, 0)
(8, 2088, 0)
(8, 2092, 0)
(8, 2096, 0)
(8, 2100, 0)
(8, 2104, 0)
(8, 2108, 0)
(8, 2112, 0)
(8, 2116, 0)
(8, 2120, 0)
(8, 2124, 0)
(8, 2128, 0)
(8, 2132, 0)
(8, 2136, 0)
(8, 2140, 0)
(8, 2144, 0)
(8, 2148, 0)
(8, 2152, 0)
(8, 2156, 0)
(8, 2160, 0)
(8, 2164, 0)
(8, 2168, 0)
(8, 2172, 0)
(8, 2176, 0)
(8, 2180, 0)
(8, 2184, 0)
(8, 2188, 0)
(8, 2192, 0)
(8, 2196, 0)
(8, 2200, 0)
(8, 2204, 0)
(8, 2208, 0)
(8, 2212, 0)
(8, 2216, 0)
(8, 2220, 0)
(8, 2224, 0)
(8, 2228, 0)
(8, 2232, 0)
(8, 2236, 0)
(8, 2240, 0)
(8, 2244, 0)
(8, 2248, 0)
(8, 2252, 0)
(8, 2256, 0)
(8, 2260, 0)
(8, 2264, 0)

(16, 696, 0)
(16, 700, 0)
(16, 704, 0)
(16, 708, 0)
(16, 712, 0)
(16, 716, 0)
(16, 720, 0)
(16, 724, 0)
(16, 728, 0)
(16, 732, 0)
(16, 736, 0)
(16, 740, 0)
(16, 744, 0)
(16, 748, 0)
(16, 752, 0)
(16, 756, 0)
(16, 760, 0)
(16, 764, 0)
(16, 768, 0)
(16, 772, 0)
(16, 776, 0)
(16, 780, 0)
(16, 784, 0)
(16, 788, 0)
(16, 792, 0)
(16, 796, 0)
(16, 800, 0)
(16, 804, 0)
(16, 808, 0)
(16, 812, 0)
(16, 816, 0)
(16, 820, 0)
(16, 824, 0)
(16, 828, 0)
(16, 832, 0)
(16, 836, 0)
(16, 840, 0)
(16, 844, 0)
(16, 848, 0)
(16, 852, 0)
(16, 856, 0)
(16, 860, 0)
(16, 864, 0)
(16, 868, 0)
(16, 872, 0)
(16, 876, 0)
(16, 880, 0)
(16, 884, 0)
(16, 888, 0)
(16, 892, 0)
(16, 896, 0)
(16, 900, 0)
(16, 904, 0)
(16, 908, 0)
(16, 912, 0)
(16, 916, 0)
(16, 920, 0)
(16, 924, 0)
(16, 928, 0)
(16, 932, 0)
(16, 936, 0)
(16, 940, 0)
(16, 944, 0)
(16, 948, 0)
(16, 952, 0)
(16, 956, 0)
(16, 960, 0)
(16, 964, 0)
(16, 968, 0)
(16, 972, 0)
(16, 976, 0)
(16, 980, 0)
(16, 984, 0)
(16, 988, 0)
(16, 992, 0)
(16, 996, 0)
(16, 1000, 0

(16, 3136, 0)
(16, 3140, 0)
(16, 3144, 0)
(16, 3148, 0)
(16, 3152, 0)
(16, 3156, 0)
(16, 3160, 0)
(16, 3164, 0)
(16, 3168, 0)
(16, 3172, 0)
(16, 3176, 0)
(16, 3180, 0)
(16, 3184, 0)
(16, 3188, 0)
(16, 3192, 0)
(16, 3196, 0)
(16, 3200, 0)
(16, 3204, 0)
(16, 3208, 0)
(16, 3212, 0)
(16, 3216, 0)
(16, 3220, 0)
(16, 3224, 0)
(16, 3228, 0)
(16, 3232, 0)
(16, 3236, 0)
(16, 3240, 0)
(16, 3244, 0)
(16, 3248, 0)
(16, 3252, 0)
(16, 3256, 0)
(16, 3260, 0)
(16, 3264, 0)
(16, 3268, 0)
(16, 3272, 0)
(16, 3276, 0)
(16, 3280, 0)
(16, 3284, 0)
(16, 3288, 0)
(16, 3292, 0)
(16, 3296, 0)
(16, 3300, 0)
(16, 3304, 0)
(16, 3308, 0)
(16, 3312, 0)
(16, 3316, 0)
(16, 3320, 0)
(16, 3324, 0)
(16, 3328, 0)
(16, 3332, 0)
(16, 3336, 0)
(16, 3340, 0)
(16, 3344, 0)
(16, 3348, 0)
(16, 3352, 0)
(16, 3356, 0)
(16, 3360, 0)
(16, 3364, 0)
(16, 3368, 0)
(16, 3372, 0)
(16, 3376, 0)
(16, 3380, 0)
(16, 3384, 0)
(16, 3388, 0)
(16, 3392, 0)
(16, 3396, 0)
(16, 3400, 0)
(16, 3404, 0)
(16, 3408, 0)
(16, 3412, 0)
(16, 3416, 0)
(16, 3

(32, 1520, 0)
(32, 1524, 0)
(32, 1528, 0)
(32, 1532, 0)
(32, 1536, 0)
(32, 1540, 0)
(32, 1544, 0)
(32, 1548, 0)
(32, 1552, 0)
(32, 1556, 0)
(32, 1560, 0)
(32, 1564, 0)
(32, 1568, 0)
(32, 1572, 0)
(32, 1576, 0)
(32, 1580, 0)
(32, 1584, 0)
(32, 1588, 0)
(32, 1592, 0)
(32, 1596, 0)
(32, 1600, 0)
(32, 1604, 0)
(32, 1608, 0)
(32, 1612, 0)
(32, 1616, 0)
(32, 1620, 0)
(32, 1624, 0)
(32, 1628, 0)
(32, 1632, 0)
(32, 1636, 0)
(32, 1640, 0)
(32, 1644, 0)
(32, 1648, 0)
(32, 1652, 0)
(32, 1656, 0)
(32, 1660, 0)
(32, 1664, 0)
(32, 1668, 0)
(32, 1672, 0)
(32, 1676, 0)
(32, 1680, 0)
(32, 1684, 0)
(32, 1688, 0)
(32, 1692, 0)
(32, 1696, 0)
(32, 1700, 0)
(32, 1704, 0)
(32, 1708, 0)
(32, 1712, 0)
(32, 1716, 0)
(32, 1720, 0)
(32, 1724, 0)
(32, 1728, 0)
(32, 1732, 0)
(32, 1736, 0)
(32, 1740, 0)
(32, 1744, 0)
(32, 1748, 0)
(32, 1752, 0)
(32, 1756, 0)
(32, 1760, 0)
(32, 1764, 0)
(32, 1768, 0)
(32, 1772, 0)
(32, 1776, 0)
(32, 1780, 0)
(32, 1784, 0)
(32, 1788, 0)
(32, 1792, 0)
(32, 1796, 0)
(32, 1800, 0)
(32, 1

(32, 4080, 0)
(32, 4084, 0)
(32, 4088, 0)
(32, 4092, 0)
(32, 4096, 0)
(32, 4100, 0)
(64, 4, 0)
(64, 8, 0)
(64, 12, 0)
(64, 16, 0)
(64, 20, 0)
(64, 24, 0)
(64, 28, 0)
(64, 32, 0)
(64, 36, 0)
(64, 40, 0)
(64, 44, 0)
(64, 48, 0)
(64, 52, 0)
(64, 56, 0)
(64, 60, 0)
(64, 64, 0)
(64, 68, 0)
(64, 72, 0)
(64, 76, 0)
(64, 80, 0)
(64, 84, 0)
(64, 88, 0)
(64, 92, 0)
(64, 96, 0)
(64, 100, 0)
(64, 104, 0)
(64, 108, 0)
(64, 112, 0)
(64, 116, 0)
(64, 120, 0)
(64, 124, 0)
(64, 128, 0)
(64, 132, 0)
(64, 136, 0)
(64, 140, 0)
(64, 144, 0)
(64, 148, 0)
(64, 152, 0)
(64, 156, 0)
(64, 160, 0)
(64, 164, 0)
(64, 168, 0)
(64, 172, 0)
(64, 176, 0)
(64, 180, 0)
(64, 184, 0)
(64, 188, 0)
(64, 192, 0)
(64, 196, 0)
(64, 200, 0)
(64, 204, 0)
(64, 208, 0)
(64, 212, 0)
(64, 216, 0)
(64, 220, 0)
(64, 224, 0)
(64, 228, 0)
(64, 232, 0)
(64, 236, 0)
(64, 240, 0)
(64, 244, 0)
(64, 248, 0)
(64, 252, 0)
(64, 256, 0)
(64, 260, 0)
(64, 264, 0)
(64, 268, 0)
(64, 272, 0)
(64, 276, 0)
(64, 280, 0)
(64, 284, 0)
(64, 288, 0)
(64, 2

(64, 2476, 0)
(64, 2480, 0)
(64, 2484, 0)
(64, 2488, 0)
(64, 2492, 0)
(64, 2496, 0)
(64, 2500, 0)
(64, 2504, 0)
(64, 2508, 0)
(64, 2512, 0)
(64, 2516, 0)
(64, 2520, 0)
(64, 2524, 0)
(64, 2528, 0)
(64, 2532, 0)
(64, 2536, 0)
(64, 2540, 0)
(64, 2544, 0)
(64, 2548, 0)
(64, 2552, 0)
(64, 2556, 0)
(64, 2560, 0)
(64, 2564, 0)
(64, 2568, 0)
(64, 2572, 0)
(64, 2576, 0)
(64, 2580, 0)
(64, 2584, 0)
(64, 2588, 0)
(64, 2592, 0)
(64, 2596, 0)
(64, 2600, 0)
(64, 2604, 0)
(64, 2608, 0)
(64, 2612, 0)
(64, 2616, 0)
(64, 2620, 0)
(64, 2624, 0)
(64, 2628, 0)
(64, 2632, 0)
(64, 2636, 0)
(64, 2640, 0)
(64, 2644, 0)
(64, 2648, 0)
(64, 2652, 0)
(64, 2656, 0)
(64, 2660, 0)
(64, 2664, 0)
(64, 2668, 0)
(64, 2672, 0)
(64, 2676, 0)
(64, 2680, 0)
(64, 2684, 0)
(64, 2688, 0)
(64, 2692, 0)
(64, 2696, 0)
(64, 2700, 0)
(64, 2704, 0)
(64, 2708, 0)
(64, 2712, 0)
(64, 2716, 0)
(64, 2720, 0)
(64, 2724, 0)
(64, 2728, 0)
(64, 2732, 0)
(64, 2736, 0)
(64, 2740, 0)
(64, 2744, 0)
(64, 2748, 0)
(64, 2752, 0)
(64, 2756, 0)
(64, 2

In [95]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
mp=0
data=[]
p=2
tests=[]
tupls=[]
dk=1
for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dk: # = 1, so k = 2^b & b = ord_2(k) for some b.
            poly=D(s[h][1])
            for m in [3..4100]:
                if ZZ(mod(m,p))==mp: # mp = 0 => m is even
                    #dm=ZZ(m/p).digits(p)
                    dm=digitsum(p,m)
                    a=order(p,m)
                    mp2=mod(m,p^2) # p = 2, m even =>  mp2 = 0 or 2.
                    if mp2==2: # => m  = 2 (mod 4) & a = ord_2(m) = 1.
                            polym=evaluate(poly,m)
                            o=order(p,polym)
                            test=o-((a+6)*k+1)
                            tupl=(k,m,test)
                            tests=tests+[test]
                            print(tupl)
                            tupls=tupls+[tupl]
ut=uniteList(tests);print();print("tests:",ut)

(1, 6, 0)
(1, 10, 0)
(1, 14, 0)
(1, 18, 0)
(1, 22, 0)
(1, 26, 0)
(1, 30, 0)
(1, 34, 0)
(1, 38, 0)
(1, 42, 0)
(1, 46, 0)
(1, 50, 0)
(1, 54, 0)
(1, 58, 0)
(1, 62, 0)
(1, 66, 0)
(1, 70, 0)
(1, 74, 0)
(1, 78, 0)
(1, 82, 0)
(1, 86, 0)
(1, 90, 0)
(1, 94, 0)
(1, 98, 0)
(1, 102, 0)
(1, 106, 0)
(1, 110, 0)
(1, 114, 0)
(1, 118, 0)
(1, 122, 0)
(1, 126, 0)
(1, 130, 0)
(1, 134, 0)
(1, 138, 0)
(1, 142, 0)
(1, 146, 0)
(1, 150, 0)
(1, 154, 0)
(1, 158, 0)
(1, 162, 0)
(1, 166, 0)
(1, 170, 0)
(1, 174, 0)
(1, 178, 0)
(1, 182, 0)
(1, 186, 0)
(1, 190, 0)
(1, 194, 0)
(1, 198, 0)
(1, 202, 0)
(1, 206, 0)
(1, 210, 0)
(1, 214, 0)
(1, 218, 0)
(1, 222, 0)
(1, 226, 0)
(1, 230, 0)
(1, 234, 0)
(1, 238, 0)
(1, 242, 0)
(1, 246, 0)
(1, 250, 0)
(1, 254, 0)
(1, 258, 0)
(1, 262, 0)
(1, 266, 0)
(1, 270, 0)
(1, 274, 0)
(1, 278, 0)
(1, 282, 0)
(1, 286, 0)
(1, 290, 0)
(1, 294, 0)
(1, 298, 0)
(1, 302, 0)
(1, 306, 0)
(1, 310, 0)
(1, 314, 0)
(1, 318, 0)
(1, 322, 0)
(1, 326, 0)
(1, 330, 0)
(1, 334, 0)
(1, 338, 0)
(1, 342, 0)
(1, 3

(1, 3530, 0)
(1, 3534, 0)
(1, 3538, 0)
(1, 3542, 0)
(1, 3546, 0)
(1, 3550, 0)
(1, 3554, 0)
(1, 3558, 0)
(1, 3562, 0)
(1, 3566, 0)
(1, 3570, 0)
(1, 3574, 0)
(1, 3578, 0)
(1, 3582, 0)
(1, 3586, 0)
(1, 3590, 0)
(1, 3594, 0)
(1, 3598, 0)
(1, 3602, 0)
(1, 3606, 0)
(1, 3610, 0)
(1, 3614, 0)
(1, 3618, 0)
(1, 3622, 0)
(1, 3626, 0)
(1, 3630, 0)
(1, 3634, 0)
(1, 3638, 0)
(1, 3642, 0)
(1, 3646, 0)
(1, 3650, 0)
(1, 3654, 0)
(1, 3658, 0)
(1, 3662, 0)
(1, 3666, 0)
(1, 3670, 0)
(1, 3674, 0)
(1, 3678, 0)
(1, 3682, 0)
(1, 3686, 0)
(1, 3690, 0)
(1, 3694, 0)
(1, 3698, 0)
(1, 3702, 0)
(1, 3706, 0)
(1, 3710, 0)
(1, 3714, 0)
(1, 3718, 0)
(1, 3722, 0)
(1, 3726, 0)
(1, 3730, 0)
(1, 3734, 0)
(1, 3738, 0)
(1, 3742, 0)
(1, 3746, 0)
(1, 3750, 0)
(1, 3754, 0)
(1, 3758, 0)
(1, 3762, 0)
(1, 3766, 0)
(1, 3770, 0)
(1, 3774, 0)
(1, 3778, 0)
(1, 3782, 0)
(1, 3786, 0)
(1, 3790, 0)
(1, 3794, 0)
(1, 3798, 0)
(1, 3802, 0)
(1, 3806, 0)
(1, 3810, 0)
(1, 3814, 0)
(1, 3818, 0)
(1, 3822, 0)
(1, 3826, 0)
(1, 3830, 0)
(1, 3834, 0)

(2, 3722, 0)
(2, 3726, 0)
(2, 3730, 0)
(2, 3734, 0)
(2, 3738, 0)
(2, 3742, 0)
(2, 3746, 0)
(2, 3750, 0)
(2, 3754, 0)
(2, 3758, 0)
(2, 3762, 0)
(2, 3766, 0)
(2, 3770, 0)
(2, 3774, 0)
(2, 3778, 0)
(2, 3782, 0)
(2, 3786, 0)
(2, 3790, 0)
(2, 3794, 0)
(2, 3798, 0)
(2, 3802, 0)
(2, 3806, 0)
(2, 3810, 0)
(2, 3814, 0)
(2, 3818, 0)
(2, 3822, 0)
(2, 3826, 0)
(2, 3830, 0)
(2, 3834, 0)
(2, 3838, 0)
(2, 3842, 0)
(2, 3846, 0)
(2, 3850, 0)
(2, 3854, 0)
(2, 3858, 0)
(2, 3862, 0)
(2, 3866, 0)
(2, 3870, 0)
(2, 3874, 0)
(2, 3878, 0)
(2, 3882, 0)
(2, 3886, 0)
(2, 3890, 0)
(2, 3894, 0)
(2, 3898, 0)
(2, 3902, 0)
(2, 3906, 0)
(2, 3910, 0)
(2, 3914, 0)
(2, 3918, 0)
(2, 3922, 0)
(2, 3926, 0)
(2, 3930, 0)
(2, 3934, 0)
(2, 3938, 0)
(2, 3942, 0)
(2, 3946, 0)
(2, 3950, 0)
(2, 3954, 0)
(2, 3958, 0)
(2, 3962, 0)
(2, 3966, 0)
(2, 3970, 0)
(2, 3974, 0)
(2, 3978, 0)
(2, 3982, 0)
(2, 3986, 0)
(2, 3990, 0)
(2, 3994, 0)
(2, 3998, 0)
(2, 4002, 0)
(2, 4006, 0)
(2, 4010, 0)
(2, 4014, 0)
(2, 4018, 0)
(2, 4022, 0)
(2, 4026, 0)

(4, 2846, 0)
(4, 2850, 0)
(4, 2854, 0)
(4, 2858, 0)
(4, 2862, 0)
(4, 2866, 0)
(4, 2870, 0)
(4, 2874, 0)
(4, 2878, 0)
(4, 2882, 0)
(4, 2886, 0)
(4, 2890, 0)
(4, 2894, 0)
(4, 2898, 0)
(4, 2902, 0)
(4, 2906, 0)
(4, 2910, 0)
(4, 2914, 0)
(4, 2918, 0)
(4, 2922, 0)
(4, 2926, 0)
(4, 2930, 0)
(4, 2934, 0)
(4, 2938, 0)
(4, 2942, 0)
(4, 2946, 0)
(4, 2950, 0)
(4, 2954, 0)
(4, 2958, 0)
(4, 2962, 0)
(4, 2966, 0)
(4, 2970, 0)
(4, 2974, 0)
(4, 2978, 0)
(4, 2982, 0)
(4, 2986, 0)
(4, 2990, 0)
(4, 2994, 0)
(4, 2998, 0)
(4, 3002, 0)
(4, 3006, 0)
(4, 3010, 0)
(4, 3014, 0)
(4, 3018, 0)
(4, 3022, 0)
(4, 3026, 0)
(4, 3030, 0)
(4, 3034, 0)
(4, 3038, 0)
(4, 3042, 0)
(4, 3046, 0)
(4, 3050, 0)
(4, 3054, 0)
(4, 3058, 0)
(4, 3062, 0)
(4, 3066, 0)
(4, 3070, 0)
(4, 3074, 0)
(4, 3078, 0)
(4, 3082, 0)
(4, 3086, 0)
(4, 3090, 0)
(4, 3094, 0)
(4, 3098, 0)
(4, 3102, 0)
(4, 3106, 0)
(4, 3110, 0)
(4, 3114, 0)
(4, 3118, 0)
(4, 3122, 0)
(4, 3126, 0)
(4, 3130, 0)
(4, 3134, 0)
(4, 3138, 0)
(4, 3142, 0)
(4, 3146, 0)
(4, 3150, 0)

(8, 2286, 0)
(8, 2290, 0)
(8, 2294, 0)
(8, 2298, 0)
(8, 2302, 0)
(8, 2306, 0)
(8, 2310, 0)
(8, 2314, 0)
(8, 2318, 0)
(8, 2322, 0)
(8, 2326, 0)
(8, 2330, 0)
(8, 2334, 0)
(8, 2338, 0)
(8, 2342, 0)
(8, 2346, 0)
(8, 2350, 0)
(8, 2354, 0)
(8, 2358, 0)
(8, 2362, 0)
(8, 2366, 0)
(8, 2370, 0)
(8, 2374, 0)
(8, 2378, 0)
(8, 2382, 0)
(8, 2386, 0)
(8, 2390, 0)
(8, 2394, 0)
(8, 2398, 0)
(8, 2402, 0)
(8, 2406, 0)
(8, 2410, 0)
(8, 2414, 0)
(8, 2418, 0)
(8, 2422, 0)
(8, 2426, 0)
(8, 2430, 0)
(8, 2434, 0)
(8, 2438, 0)
(8, 2442, 0)
(8, 2446, 0)
(8, 2450, 0)
(8, 2454, 0)
(8, 2458, 0)
(8, 2462, 0)
(8, 2466, 0)
(8, 2470, 0)
(8, 2474, 0)
(8, 2478, 0)
(8, 2482, 0)
(8, 2486, 0)
(8, 2490, 0)
(8, 2494, 0)
(8, 2498, 0)
(8, 2502, 0)
(8, 2506, 0)
(8, 2510, 0)
(8, 2514, 0)
(8, 2518, 0)
(8, 2522, 0)
(8, 2526, 0)
(8, 2530, 0)
(8, 2534, 0)
(8, 2538, 0)
(8, 2542, 0)
(8, 2546, 0)
(8, 2550, 0)
(8, 2554, 0)
(8, 2558, 0)
(8, 2562, 0)
(8, 2566, 0)
(8, 2570, 0)
(8, 2574, 0)
(8, 2578, 0)
(8, 2582, 0)
(8, 2586, 0)
(8, 2590, 0)

(16, 890, 0)
(16, 894, 0)
(16, 898, 0)
(16, 902, 0)
(16, 906, 0)
(16, 910, 0)
(16, 914, 0)
(16, 918, 0)
(16, 922, 0)
(16, 926, 0)
(16, 930, 0)
(16, 934, 0)
(16, 938, 0)
(16, 942, 0)
(16, 946, 0)
(16, 950, 0)
(16, 954, 0)
(16, 958, 0)
(16, 962, 0)
(16, 966, 0)
(16, 970, 0)
(16, 974, 0)
(16, 978, 0)
(16, 982, 0)
(16, 986, 0)
(16, 990, 0)
(16, 994, 0)
(16, 998, 0)
(16, 1002, 0)
(16, 1006, 0)
(16, 1010, 0)
(16, 1014, 0)
(16, 1018, 0)
(16, 1022, 0)
(16, 1026, 0)
(16, 1030, 0)
(16, 1034, 0)
(16, 1038, 0)
(16, 1042, 0)
(16, 1046, 0)
(16, 1050, 0)
(16, 1054, 0)
(16, 1058, 0)
(16, 1062, 0)
(16, 1066, 0)
(16, 1070, 0)
(16, 1074, 0)
(16, 1078, 0)
(16, 1082, 0)
(16, 1086, 0)
(16, 1090, 0)
(16, 1094, 0)
(16, 1098, 0)
(16, 1102, 0)
(16, 1106, 0)
(16, 1110, 0)
(16, 1114, 0)
(16, 1118, 0)
(16, 1122, 0)
(16, 1126, 0)
(16, 1130, 0)
(16, 1134, 0)
(16, 1138, 0)
(16, 1142, 0)
(16, 1146, 0)
(16, 1150, 0)
(16, 1154, 0)
(16, 1158, 0)
(16, 1162, 0)
(16, 1166, 0)
(16, 1170, 0)
(16, 1174, 0)
(16, 1178, 0)
(16, 1

(16, 3294, 0)
(16, 3298, 0)
(16, 3302, 0)
(16, 3306, 0)
(16, 3310, 0)
(16, 3314, 0)
(16, 3318, 0)
(16, 3322, 0)
(16, 3326, 0)
(16, 3330, 0)
(16, 3334, 0)
(16, 3338, 0)
(16, 3342, 0)
(16, 3346, 0)
(16, 3350, 0)
(16, 3354, 0)
(16, 3358, 0)
(16, 3362, 0)
(16, 3366, 0)
(16, 3370, 0)
(16, 3374, 0)
(16, 3378, 0)
(16, 3382, 0)
(16, 3386, 0)
(16, 3390, 0)
(16, 3394, 0)
(16, 3398, 0)
(16, 3402, 0)
(16, 3406, 0)
(16, 3410, 0)
(16, 3414, 0)
(16, 3418, 0)
(16, 3422, 0)
(16, 3426, 0)
(16, 3430, 0)
(16, 3434, 0)
(16, 3438, 0)
(16, 3442, 0)
(16, 3446, 0)
(16, 3450, 0)
(16, 3454, 0)
(16, 3458, 0)
(16, 3462, 0)
(16, 3466, 0)
(16, 3470, 0)
(16, 3474, 0)
(16, 3478, 0)
(16, 3482, 0)
(16, 3486, 0)
(16, 3490, 0)
(16, 3494, 0)
(16, 3498, 0)
(16, 3502, 0)
(16, 3506, 0)
(16, 3510, 0)
(16, 3514, 0)
(16, 3518, 0)
(16, 3522, 0)
(16, 3526, 0)
(16, 3530, 0)
(16, 3534, 0)
(16, 3538, 0)
(16, 3542, 0)
(16, 3546, 0)
(16, 3550, 0)
(16, 3554, 0)
(16, 3558, 0)
(16, 3562, 0)
(16, 3566, 0)
(16, 3570, 0)
(16, 3574, 0)
(16, 3

(32, 1858, 0)
(32, 1862, 0)
(32, 1866, 0)
(32, 1870, 0)
(32, 1874, 0)
(32, 1878, 0)
(32, 1882, 0)
(32, 1886, 0)
(32, 1890, 0)
(32, 1894, 0)
(32, 1898, 0)
(32, 1902, 0)
(32, 1906, 0)
(32, 1910, 0)
(32, 1914, 0)
(32, 1918, 0)
(32, 1922, 0)
(32, 1926, 0)
(32, 1930, 0)
(32, 1934, 0)
(32, 1938, 0)
(32, 1942, 0)
(32, 1946, 0)
(32, 1950, 0)
(32, 1954, 0)
(32, 1958, 0)
(32, 1962, 0)
(32, 1966, 0)
(32, 1970, 0)
(32, 1974, 0)
(32, 1978, 0)
(32, 1982, 0)
(32, 1986, 0)
(32, 1990, 0)
(32, 1994, 0)
(32, 1998, 0)
(32, 2002, 0)
(32, 2006, 0)
(32, 2010, 0)
(32, 2014, 0)
(32, 2018, 0)
(32, 2022, 0)
(32, 2026, 0)
(32, 2030, 0)
(32, 2034, 0)
(32, 2038, 0)
(32, 2042, 0)
(32, 2046, 0)
(32, 2050, 0)
(32, 2054, 0)
(32, 2058, 0)
(32, 2062, 0)
(32, 2066, 0)
(32, 2070, 0)
(32, 2074, 0)
(32, 2078, 0)
(32, 2082, 0)
(32, 2086, 0)
(32, 2090, 0)
(32, 2094, 0)
(32, 2098, 0)
(32, 2102, 0)
(32, 2106, 0)
(32, 2110, 0)
(32, 2114, 0)
(32, 2118, 0)
(32, 2122, 0)
(32, 2126, 0)
(32, 2130, 0)
(32, 2134, 0)
(32, 2138, 0)
(32, 2

(64, 206, 0)
(64, 210, 0)
(64, 214, 0)
(64, 218, 0)
(64, 222, 0)
(64, 226, 0)
(64, 230, 0)
(64, 234, 0)
(64, 238, 0)
(64, 242, 0)
(64, 246, 0)
(64, 250, 0)
(64, 254, 0)
(64, 258, 0)
(64, 262, 0)
(64, 266, 0)
(64, 270, 0)
(64, 274, 0)
(64, 278, 0)
(64, 282, 0)
(64, 286, 0)
(64, 290, 0)
(64, 294, 0)
(64, 298, 0)
(64, 302, 0)
(64, 306, 0)
(64, 310, 0)
(64, 314, 0)
(64, 318, 0)
(64, 322, 0)
(64, 326, 0)
(64, 330, 0)
(64, 334, 0)
(64, 338, 0)
(64, 342, 0)
(64, 346, 0)
(64, 350, 0)
(64, 354, 0)
(64, 358, 0)
(64, 362, 0)
(64, 366, 0)
(64, 370, 0)
(64, 374, 0)
(64, 378, 0)
(64, 382, 0)
(64, 386, 0)
(64, 390, 0)
(64, 394, 0)
(64, 398, 0)
(64, 402, 0)
(64, 406, 0)
(64, 410, 0)
(64, 414, 0)
(64, 418, 0)
(64, 422, 0)
(64, 426, 0)
(64, 430, 0)
(64, 434, 0)
(64, 438, 0)
(64, 442, 0)
(64, 446, 0)
(64, 450, 0)
(64, 454, 0)
(64, 458, 0)
(64, 462, 0)
(64, 466, 0)
(64, 470, 0)
(64, 474, 0)
(64, 478, 0)
(64, 482, 0)
(64, 486, 0)
(64, 490, 0)
(64, 494, 0)
(64, 498, 0)
(64, 502, 0)
(64, 506, 0)
(64, 510, 0)

(64, 2638, 0)
(64, 2642, 0)
(64, 2646, 0)
(64, 2650, 0)
(64, 2654, 0)
(64, 2658, 0)
(64, 2662, 0)
(64, 2666, 0)
(64, 2670, 0)
(64, 2674, 0)
(64, 2678, 0)
(64, 2682, 0)
(64, 2686, 0)
(64, 2690, 0)
(64, 2694, 0)
(64, 2698, 0)
(64, 2702, 0)
(64, 2706, 0)
(64, 2710, 0)
(64, 2714, 0)
(64, 2718, 0)
(64, 2722, 0)
(64, 2726, 0)
(64, 2730, 0)
(64, 2734, 0)
(64, 2738, 0)
(64, 2742, 0)
(64, 2746, 0)
(64, 2750, 0)
(64, 2754, 0)
(64, 2758, 0)
(64, 2762, 0)
(64, 2766, 0)
(64, 2770, 0)
(64, 2774, 0)
(64, 2778, 0)
(64, 2782, 0)
(64, 2786, 0)
(64, 2790, 0)
(64, 2794, 0)
(64, 2798, 0)
(64, 2802, 0)
(64, 2806, 0)
(64, 2810, 0)
(64, 2814, 0)
(64, 2818, 0)
(64, 2822, 0)
(64, 2826, 0)
(64, 2830, 0)
(64, 2834, 0)
(64, 2838, 0)
(64, 2842, 0)
(64, 2846, 0)
(64, 2850, 0)
(64, 2854, 0)
(64, 2858, 0)
(64, 2862, 0)
(64, 2866, 0)
(64, 2870, 0)
(64, 2874, 0)
(64, 2878, 0)
(64, 2882, 0)
(64, 2886, 0)
(64, 2890, 0)
(64, 2894, 0)
(64, 2898, 0)
(64, 2902, 0)
(64, 2906, 0)
(64, 2910, 0)
(64, 2914, 0)
(64, 2918, 0)
(64, 2

In [95]:
order(2,0)

+Infinity

In [25]:
import time
pairs=[]
start=time.time()
count=0
for n in [1..4*10^4]:
    if mod(n,10^4)==0:
        print((n/10^4,time.time()-start))
    cn=catalan_number(n)
    if order(2,cn)==1:
        count=count+1
        pair=(count,cn)
        pairs=pairs+[pair]
import pickle
wfile = open('/Users/barrybrent/12dec22no1.txt','wb')
t = pickle.dumps(str(pairs))
wfile.write(t)
wfile.close()
print("length:",len(pairs))

(1, 4.75567102432251)
(2, 23.339736938476562)
(3, 60.761170864105225)
(4, 120.3875687122345)
length: 118


In [34]:
rvfile = open('/Users/barrybrent/12dec22no1.txt','rb')
rv=pickle.load(rvfile)
rvfile.close()
v=stripQuotationMarks(rv)
ln=len(v)
last=v[ln-1]
print(last);print()
print(order(2,last[1]))

(118, ...)

1


In [22]:
rvfile = open('/Users/barrybrent/12dec22no1.txt','rb')
rv=pickle.load(rvfile)
rvfile.close()
v=stripQuotationMarks(rv)

def c1(n):return v[n][1]
data=[]
for j in [0..len(v)-1]:
    data=data+[c1(j)-v[j][1]]
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [18]:
rvfile = open('/Users/barrybrent/12dec22no1.txt','rb')
rv=pickle.load(rvfile)
rvfile.close()
v=stripQuotationMarks(rv)

def c1(n):return v[n][1]
data=[]
for j in [0..3]:
    data=data+[c1(j)]
print(data)

[2, 14, 42, 1430]


In [52]:
import pickle
rsfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

rvfile = open('/Users/barrybrent/12dec22no1.txt','rb')
rv=pickle.load(rvfile)
rvfile.close()
v=stripQuotationMarks(rv)

def c1(n):return v[n][1]

mp2=0
data=[]
p=2
tests2=[]
t_set=[]
no=[]
dk=2
skips=[]
#dkd=dk.digits(p)
for h in [0..len(s)-1]:
    k=s[h][0]
    if digitsum(p,k)==dk:
        poly=D(s[h][1])
        for m in [3..1003]:
                if mp2==mod(m,p^2):
                    polym=evaluate(poly,m)
                    o=order(p,polym)
                    a=order(p,m)
                   #t=(6*k+a*k+2-o)/4
                    t=(k*(a+6)+2-o)/4
                    if len(t_set)>0:
                        if t< max(t_set):no=no+[t]
                    t_set=t_set+[t]
                    cnt=catalan_number(t)
                    o2cnt=order(2,cnt)
                    tupl=(k,m,t,o2cnt)
                    tests2=tests2+[o2cnt]
                    print(tupl)
print();print("How many decreases in the t sequence?: ",len(no));print()
u2=uniteList(tests2);print("set union of 2-orders:",u2);print()
u4=uniteList(t_set);print("set union of t_set:");print(u4);print()
print("min(set union of 2-orders):",min(u4))
print("max(set union of 2-orders):",max(u4))
for j in [min(u4)..max(u4)]:
    cj=catalan_number(j)
    if order(2,cj)==1:
        if (j in u4)==False:
            skips=skips+[j]
print("skips:" ,skips)
# u4 in OEIS:
# Search: seq:2,4,5,8,9,11,16,17,19,23,32,33,35,39,47,64,65,67,71,79,95 
# Displaying 1-1 of 1 result found.	page 1
# A099628		Numbers m where m-th Catalan number A000108(m) = binomial(2m,m)/(m+1) 
# is divisible by 2 but not by 4

(3, 4, 2, 1)
(3, 8, 2, 1)
(3, 12, 2, 1)
(3, 16, 2, 1)
(3, 20, 2, 1)
(3, 24, 2, 1)
(3, 28, 2, 1)
(3, 32, 2, 1)
(3, 36, 2, 1)
(3, 40, 2, 1)
(3, 44, 2, 1)
(3, 48, 2, 1)
(3, 52, 2, 1)
(3, 56, 2, 1)
(3, 60, 2, 1)
(3, 64, 2, 1)
(3, 68, 2, 1)
(3, 72, 2, 1)
(3, 76, 2, 1)
(3, 80, 2, 1)
(3, 84, 2, 1)
(3, 88, 2, 1)
(3, 92, 2, 1)
(3, 96, 2, 1)
(3, 100, 2, 1)
(3, 104, 2, 1)
(3, 108, 2, 1)
(3, 112, 2, 1)
(3, 116, 2, 1)
(3, 120, 2, 1)
(3, 124, 2, 1)
(3, 128, 2, 1)
(3, 132, 2, 1)
(3, 136, 2, 1)
(3, 140, 2, 1)
(3, 144, 2, 1)
(3, 148, 2, 1)
(3, 152, 2, 1)
(3, 156, 2, 1)
(3, 160, 2, 1)
(3, 164, 2, 1)
(3, 168, 2, 1)
(3, 172, 2, 1)
(3, 176, 2, 1)
(3, 180, 2, 1)
(3, 184, 2, 1)
(3, 188, 2, 1)
(3, 192, 2, 1)
(3, 196, 2, 1)
(3, 200, 2, 1)
(3, 204, 2, 1)
(3, 208, 2, 1)
(3, 212, 2, 1)
(3, 216, 2, 1)
(3, 220, 2, 1)
(3, 224, 2, 1)
(3, 228, 2, 1)
(3, 232, 2, 1)
(3, 236, 2, 1)
(3, 240, 2, 1)
(3, 244, 2, 1)
(3, 248, 2, 1)
(3, 252, 2, 1)
(3, 256, 2, 1)
(3, 260, 2, 1)
(3, 264, 2, 1)
(3, 268, 2, 1)
(3, 272, 2, 1)
(3, 27

(9, 300, 8, 1)
(9, 304, 8, 1)
(9, 308, 8, 1)
(9, 312, 8, 1)
(9, 316, 8, 1)
(9, 320, 8, 1)
(9, 324, 8, 1)
(9, 328, 8, 1)
(9, 332, 8, 1)
(9, 336, 8, 1)
(9, 340, 8, 1)
(9, 344, 8, 1)
(9, 348, 8, 1)
(9, 352, 8, 1)
(9, 356, 8, 1)
(9, 360, 8, 1)
(9, 364, 8, 1)
(9, 368, 8, 1)
(9, 372, 8, 1)
(9, 376, 8, 1)
(9, 380, 8, 1)
(9, 384, 8, 1)
(9, 388, 8, 1)
(9, 392, 8, 1)
(9, 396, 8, 1)
(9, 400, 8, 1)
(9, 404, 8, 1)
(9, 408, 8, 1)
(9, 412, 8, 1)
(9, 416, 8, 1)
(9, 420, 8, 1)
(9, 424, 8, 1)
(9, 428, 8, 1)
(9, 432, 8, 1)
(9, 436, 8, 1)
(9, 440, 8, 1)
(9, 444, 8, 1)
(9, 448, 8, 1)
(9, 452, 8, 1)
(9, 456, 8, 1)
(9, 460, 8, 1)
(9, 464, 8, 1)
(9, 468, 8, 1)
(9, 472, 8, 1)
(9, 476, 8, 1)
(9, 480, 8, 1)
(9, 484, 8, 1)
(9, 488, 8, 1)
(9, 492, 8, 1)
(9, 496, 8, 1)
(9, 500, 8, 1)
(9, 504, 8, 1)
(9, 508, 8, 1)
(9, 512, 8, 1)
(9, 516, 8, 1)
(9, 520, 8, 1)
(9, 524, 8, 1)
(9, 528, 8, 1)
(9, 532, 8, 1)
(9, 536, 8, 1)
(9, 540, 8, 1)
(9, 544, 8, 1)
(9, 548, 8, 1)
(9, 552, 8, 1)
(9, 556, 8, 1)
(9, 560, 8, 1)
(9, 564, 8

(12, 960, 11, 1)
(12, 964, 11, 1)
(12, 968, 11, 1)
(12, 972, 11, 1)
(12, 976, 11, 1)
(12, 980, 11, 1)
(12, 984, 11, 1)
(12, 988, 11, 1)
(12, 992, 11, 1)
(12, 996, 11, 1)
(12, 1000, 11, 1)
(17, 4, 16, 1)
(17, 8, 16, 1)
(17, 12, 16, 1)
(17, 16, 16, 1)
(17, 20, 16, 1)
(17, 24, 16, 1)
(17, 28, 16, 1)
(17, 32, 16, 1)
(17, 36, 16, 1)
(17, 40, 16, 1)
(17, 44, 16, 1)
(17, 48, 16, 1)
(17, 52, 16, 1)
(17, 56, 16, 1)
(17, 60, 16, 1)
(17, 64, 16, 1)
(17, 68, 16, 1)
(17, 72, 16, 1)
(17, 76, 16, 1)
(17, 80, 16, 1)
(17, 84, 16, 1)
(17, 88, 16, 1)
(17, 92, 16, 1)
(17, 96, 16, 1)
(17, 100, 16, 1)
(17, 104, 16, 1)
(17, 108, 16, 1)
(17, 112, 16, 1)
(17, 116, 16, 1)
(17, 120, 16, 1)
(17, 124, 16, 1)
(17, 128, 16, 1)
(17, 132, 16, 1)
(17, 136, 16, 1)
(17, 140, 16, 1)
(17, 144, 16, 1)
(17, 148, 16, 1)
(17, 152, 16, 1)
(17, 156, 16, 1)
(17, 160, 16, 1)
(17, 164, 16, 1)
(17, 168, 16, 1)
(17, 172, 16, 1)
(17, 176, 16, 1)
(17, 180, 16, 1)
(17, 184, 16, 1)
(17, 188, 16, 1)
(17, 192, 16, 1)
(17, 196, 16, 1)
(17, 

(20, 128, 19, 1)
(20, 132, 19, 1)
(20, 136, 19, 1)
(20, 140, 19, 1)
(20, 144, 19, 1)
(20, 148, 19, 1)
(20, 152, 19, 1)
(20, 156, 19, 1)
(20, 160, 19, 1)
(20, 164, 19, 1)
(20, 168, 19, 1)
(20, 172, 19, 1)
(20, 176, 19, 1)
(20, 180, 19, 1)
(20, 184, 19, 1)
(20, 188, 19, 1)
(20, 192, 19, 1)
(20, 196, 19, 1)
(20, 200, 19, 1)
(20, 204, 19, 1)
(20, 208, 19, 1)
(20, 212, 19, 1)
(20, 216, 19, 1)
(20, 220, 19, 1)
(20, 224, 19, 1)
(20, 228, 19, 1)
(20, 232, 19, 1)
(20, 236, 19, 1)
(20, 240, 19, 1)
(20, 244, 19, 1)
(20, 248, 19, 1)
(20, 252, 19, 1)
(20, 256, 19, 1)
(20, 260, 19, 1)
(20, 264, 19, 1)
(20, 268, 19, 1)
(20, 272, 19, 1)
(20, 276, 19, 1)
(20, 280, 19, 1)
(20, 284, 19, 1)
(20, 288, 19, 1)
(20, 292, 19, 1)
(20, 296, 19, 1)
(20, 300, 19, 1)
(20, 304, 19, 1)
(20, 308, 19, 1)
(20, 312, 19, 1)
(20, 316, 19, 1)
(20, 320, 19, 1)
(20, 324, 19, 1)
(20, 328, 19, 1)
(20, 332, 19, 1)
(20, 336, 19, 1)
(20, 340, 19, 1)
(20, 344, 19, 1)
(20, 348, 19, 1)
(20, 352, 19, 1)
(20, 356, 19, 1)
(20, 360, 19, 

(33, 204, 32, 1)
(33, 208, 32, 1)
(33, 212, 32, 1)
(33, 216, 32, 1)
(33, 220, 32, 1)
(33, 224, 32, 1)
(33, 228, 32, 1)
(33, 232, 32, 1)
(33, 236, 32, 1)
(33, 240, 32, 1)
(33, 244, 32, 1)
(33, 248, 32, 1)
(33, 252, 32, 1)
(33, 256, 32, 1)
(33, 260, 32, 1)
(33, 264, 32, 1)
(33, 268, 32, 1)
(33, 272, 32, 1)
(33, 276, 32, 1)
(33, 280, 32, 1)
(33, 284, 32, 1)
(33, 288, 32, 1)
(33, 292, 32, 1)
(33, 296, 32, 1)
(33, 300, 32, 1)
(33, 304, 32, 1)
(33, 308, 32, 1)
(33, 312, 32, 1)
(33, 316, 32, 1)
(33, 320, 32, 1)
(33, 324, 32, 1)
(33, 328, 32, 1)
(33, 332, 32, 1)
(33, 336, 32, 1)
(33, 340, 32, 1)
(33, 344, 32, 1)
(33, 348, 32, 1)
(33, 352, 32, 1)
(33, 356, 32, 1)
(33, 360, 32, 1)
(33, 364, 32, 1)
(33, 368, 32, 1)
(33, 372, 32, 1)
(33, 376, 32, 1)
(33, 380, 32, 1)
(33, 384, 32, 1)
(33, 388, 32, 1)
(33, 392, 32, 1)
(33, 396, 32, 1)
(33, 400, 32, 1)
(33, 404, 32, 1)
(33, 408, 32, 1)
(33, 412, 32, 1)
(33, 416, 32, 1)
(33, 420, 32, 1)
(33, 424, 32, 1)
(33, 428, 32, 1)
(33, 432, 32, 1)
(33, 436, 32, 

(36, 312, 35, 1)
(36, 316, 35, 1)
(36, 320, 35, 1)
(36, 324, 35, 1)
(36, 328, 35, 1)
(36, 332, 35, 1)
(36, 336, 35, 1)
(36, 340, 35, 1)
(36, 344, 35, 1)
(36, 348, 35, 1)
(36, 352, 35, 1)
(36, 356, 35, 1)
(36, 360, 35, 1)
(36, 364, 35, 1)
(36, 368, 35, 1)
(36, 372, 35, 1)
(36, 376, 35, 1)
(36, 380, 35, 1)
(36, 384, 35, 1)
(36, 388, 35, 1)
(36, 392, 35, 1)
(36, 396, 35, 1)
(36, 400, 35, 1)
(36, 404, 35, 1)
(36, 408, 35, 1)
(36, 412, 35, 1)
(36, 416, 35, 1)
(36, 420, 35, 1)
(36, 424, 35, 1)
(36, 428, 35, 1)
(36, 432, 35, 1)
(36, 436, 35, 1)
(36, 440, 35, 1)
(36, 444, 35, 1)
(36, 448, 35, 1)
(36, 452, 35, 1)
(36, 456, 35, 1)
(36, 460, 35, 1)
(36, 464, 35, 1)
(36, 468, 35, 1)
(36, 472, 35, 1)
(36, 476, 35, 1)
(36, 480, 35, 1)
(36, 484, 35, 1)
(36, 488, 35, 1)
(36, 492, 35, 1)
(36, 496, 35, 1)
(36, 500, 35, 1)
(36, 504, 35, 1)
(36, 508, 35, 1)
(36, 512, 35, 1)
(36, 516, 35, 1)
(36, 520, 35, 1)
(36, 524, 35, 1)
(36, 528, 35, 1)
(36, 532, 35, 1)
(36, 536, 35, 1)
(36, 540, 35, 1)
(36, 544, 35, 

(48, 388, 47, 1)
(48, 392, 47, 1)
(48, 396, 47, 1)
(48, 400, 47, 1)
(48, 404, 47, 1)
(48, 408, 47, 1)
(48, 412, 47, 1)
(48, 416, 47, 1)
(48, 420, 47, 1)
(48, 424, 47, 1)
(48, 428, 47, 1)
(48, 432, 47, 1)
(48, 436, 47, 1)
(48, 440, 47, 1)
(48, 444, 47, 1)
(48, 448, 47, 1)
(48, 452, 47, 1)
(48, 456, 47, 1)
(48, 460, 47, 1)
(48, 464, 47, 1)
(48, 468, 47, 1)
(48, 472, 47, 1)
(48, 476, 47, 1)
(48, 480, 47, 1)
(48, 484, 47, 1)
(48, 488, 47, 1)
(48, 492, 47, 1)
(48, 496, 47, 1)
(48, 500, 47, 1)
(48, 504, 47, 1)
(48, 508, 47, 1)
(48, 512, 47, 1)
(48, 516, 47, 1)
(48, 520, 47, 1)
(48, 524, 47, 1)
(48, 528, 47, 1)
(48, 532, 47, 1)
(48, 536, 47, 1)
(48, 540, 47, 1)
(48, 544, 47, 1)
(48, 548, 47, 1)
(48, 552, 47, 1)
(48, 556, 47, 1)
(48, 560, 47, 1)
(48, 564, 47, 1)
(48, 568, 47, 1)
(48, 572, 47, 1)
(48, 576, 47, 1)
(48, 580, 47, 1)
(48, 584, 47, 1)
(48, 588, 47, 1)
(48, 592, 47, 1)
(48, 596, 47, 1)
(48, 600, 47, 1)
(48, 604, 47, 1)
(48, 608, 47, 1)
(48, 612, 47, 1)
(48, 616, 47, 1)
(48, 620, 47, 

(66, 396, 65, 1)
(66, 400, 65, 1)
(66, 404, 65, 1)
(66, 408, 65, 1)
(66, 412, 65, 1)
(66, 416, 65, 1)
(66, 420, 65, 1)
(66, 424, 65, 1)
(66, 428, 65, 1)
(66, 432, 65, 1)
(66, 436, 65, 1)
(66, 440, 65, 1)
(66, 444, 65, 1)
(66, 448, 65, 1)
(66, 452, 65, 1)
(66, 456, 65, 1)
(66, 460, 65, 1)
(66, 464, 65, 1)
(66, 468, 65, 1)
(66, 472, 65, 1)
(66, 476, 65, 1)
(66, 480, 65, 1)
(66, 484, 65, 1)
(66, 488, 65, 1)
(66, 492, 65, 1)
(66, 496, 65, 1)
(66, 500, 65, 1)
(66, 504, 65, 1)
(66, 508, 65, 1)
(66, 512, 65, 1)
(66, 516, 65, 1)
(66, 520, 65, 1)
(66, 524, 65, 1)
(66, 528, 65, 1)
(66, 532, 65, 1)
(66, 536, 65, 1)
(66, 540, 65, 1)
(66, 544, 65, 1)
(66, 548, 65, 1)
(66, 552, 65, 1)
(66, 556, 65, 1)
(66, 560, 65, 1)
(66, 564, 65, 1)
(66, 568, 65, 1)
(66, 572, 65, 1)
(66, 576, 65, 1)
(66, 580, 65, 1)
(66, 584, 65, 1)
(66, 588, 65, 1)
(66, 592, 65, 1)
(66, 596, 65, 1)
(66, 600, 65, 1)
(66, 604, 65, 1)
(66, 608, 65, 1)
(66, 612, 65, 1)
(66, 616, 65, 1)
(66, 620, 65, 1)
(66, 624, 65, 1)
(66, 628, 65, 

(72, 352, 71, 1)
(72, 356, 71, 1)
(72, 360, 71, 1)
(72, 364, 71, 1)
(72, 368, 71, 1)
(72, 372, 71, 1)
(72, 376, 71, 1)
(72, 380, 71, 1)
(72, 384, 71, 1)
(72, 388, 71, 1)
(72, 392, 71, 1)
(72, 396, 71, 1)
(72, 400, 71, 1)
(72, 404, 71, 1)
(72, 408, 71, 1)
(72, 412, 71, 1)
(72, 416, 71, 1)
(72, 420, 71, 1)
(72, 424, 71, 1)
(72, 428, 71, 1)
(72, 432, 71, 1)
(72, 436, 71, 1)
(72, 440, 71, 1)
(72, 444, 71, 1)
(72, 448, 71, 1)
(72, 452, 71, 1)
(72, 456, 71, 1)
(72, 460, 71, 1)
(72, 464, 71, 1)
(72, 468, 71, 1)
(72, 472, 71, 1)
(72, 476, 71, 1)
(72, 480, 71, 1)
(72, 484, 71, 1)
(72, 488, 71, 1)
(72, 492, 71, 1)
(72, 496, 71, 1)
(72, 500, 71, 1)
(72, 504, 71, 1)
(72, 508, 71, 1)
(72, 512, 71, 1)
(72, 516, 71, 1)
(72, 520, 71, 1)
(72, 524, 71, 1)
(72, 528, 71, 1)
(72, 532, 71, 1)
(72, 536, 71, 1)
(72, 540, 71, 1)
(72, 544, 71, 1)
(72, 548, 71, 1)
(72, 552, 71, 1)
(72, 556, 71, 1)
(72, 560, 71, 1)
(72, 564, 71, 1)
(72, 568, 71, 1)
(72, 572, 71, 1)
(72, 576, 71, 1)
(72, 580, 71, 1)
(72, 584, 71, 

(96, 312, 95, 1)
(96, 316, 95, 1)
(96, 320, 95, 1)
(96, 324, 95, 1)
(96, 328, 95, 1)
(96, 332, 95, 1)
(96, 336, 95, 1)
(96, 340, 95, 1)
(96, 344, 95, 1)
(96, 348, 95, 1)
(96, 352, 95, 1)
(96, 356, 95, 1)
(96, 360, 95, 1)
(96, 364, 95, 1)
(96, 368, 95, 1)
(96, 372, 95, 1)
(96, 376, 95, 1)
(96, 380, 95, 1)
(96, 384, 95, 1)
(96, 388, 95, 1)
(96, 392, 95, 1)
(96, 396, 95, 1)
(96, 400, 95, 1)
(96, 404, 95, 1)
(96, 408, 95, 1)
(96, 412, 95, 1)
(96, 416, 95, 1)
(96, 420, 95, 1)
(96, 424, 95, 1)
(96, 428, 95, 1)
(96, 432, 95, 1)
(96, 436, 95, 1)
(96, 440, 95, 1)
(96, 444, 95, 1)
(96, 448, 95, 1)
(96, 452, 95, 1)
(96, 456, 95, 1)
(96, 460, 95, 1)
(96, 464, 95, 1)
(96, 468, 95, 1)
(96, 472, 95, 1)
(96, 476, 95, 1)
(96, 480, 95, 1)
(96, 484, 95, 1)
(96, 488, 95, 1)
(96, 492, 95, 1)
(96, 496, 95, 1)
(96, 500, 95, 1)
(96, 504, 95, 1)
(96, 508, 95, 1)
(96, 512, 95, 1)
(96, 516, 95, 1)
(96, 520, 95, 1)
(96, 524, 95, 1)
(96, 528, 95, 1)
(96, 532, 95, 1)
(96, 536, 95, 1)
(96, 540, 95, 1)
(96, 544, 95, 

In [51]:
import pickle
rsfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

mp2=2
data=[]
p=2
t_set=[]
dk=2
#dkd=dk.digits(p)
for h in [0..len(s)-1]:
    k=s[h][0]
    if digitsum(p,k)==dk:
        poly=D(s[h][1])
        for m in [3..1003]:
                if mp2==mod(m,p^2):
                    polym=evaluate(poly,m)
                    o=order(p,polym)
                    a=order(p,m) # a = 2 bc m = 2 (mod 4)
                    t=(a+6)*k+2-o 
                    t_set=t_set+[t]
                    tupl=(k,m,t)
                    print(tupl)
print()
print("set union of t_set:")
print(uniteList(t_set))

(3, 6, 0)
(3, 10, 0)
(3, 14, 0)
(3, 18, 0)
(3, 22, 0)
(3, 26, 0)
(3, 30, 0)
(3, 34, 0)
(3, 38, 0)
(3, 42, 0)
(3, 46, 0)
(3, 50, 0)
(3, 54, 0)
(3, 58, 0)
(3, 62, 0)
(3, 66, 0)
(3, 70, 0)
(3, 74, 0)
(3, 78, 0)
(3, 82, 0)
(3, 86, 0)
(3, 90, 0)
(3, 94, 0)
(3, 98, 0)
(3, 102, 0)
(3, 106, 0)
(3, 110, 0)
(3, 114, 0)
(3, 118, 0)
(3, 122, 0)
(3, 126, 0)
(3, 130, 0)
(3, 134, 0)
(3, 138, 0)
(3, 142, 0)
(3, 146, 0)
(3, 150, 0)
(3, 154, 0)
(3, 158, 0)
(3, 162, 0)
(3, 166, 0)
(3, 170, 0)
(3, 174, 0)
(3, 178, 0)
(3, 182, 0)
(3, 186, 0)
(3, 190, 0)
(3, 194, 0)
(3, 198, 0)
(3, 202, 0)
(3, 206, 0)
(3, 210, 0)
(3, 214, 0)
(3, 218, 0)
(3, 222, 0)
(3, 226, 0)
(3, 230, 0)
(3, 234, 0)
(3, 238, 0)
(3, 242, 0)
(3, 246, 0)
(3, 250, 0)
(3, 254, 0)
(3, 258, 0)
(3, 262, 0)
(3, 266, 0)
(3, 270, 0)
(3, 274, 0)
(3, 278, 0)
(3, 282, 0)
(3, 286, 0)
(3, 290, 0)
(3, 294, 0)
(3, 298, 0)
(3, 302, 0)
(3, 306, 0)
(3, 310, 0)
(3, 314, 0)
(3, 318, 0)
(3, 322, 0)
(3, 326, 0)
(3, 330, 0)
(3, 334, 0)
(3, 338, 0)
(3, 342, 0)
(3, 3

(9, 550, 0)
(9, 554, 0)
(9, 558, 0)
(9, 562, 0)
(9, 566, 0)
(9, 570, 0)
(9, 574, 0)
(9, 578, 0)
(9, 582, 0)
(9, 586, 0)
(9, 590, 0)
(9, 594, 0)
(9, 598, 0)
(9, 602, 0)
(9, 606, 0)
(9, 610, 0)
(9, 614, 0)
(9, 618, 0)
(9, 622, 0)
(9, 626, 0)
(9, 630, 0)
(9, 634, 0)
(9, 638, 0)
(9, 642, 0)
(9, 646, 0)
(9, 650, 0)
(9, 654, 0)
(9, 658, 0)
(9, 662, 0)
(9, 666, 0)
(9, 670, 0)
(9, 674, 0)
(9, 678, 0)
(9, 682, 0)
(9, 686, 0)
(9, 690, 0)
(9, 694, 0)
(9, 698, 0)
(9, 702, 0)
(9, 706, 0)
(9, 710, 0)
(9, 714, 0)
(9, 718, 0)
(9, 722, 0)
(9, 726, 0)
(9, 730, 0)
(9, 734, 0)
(9, 738, 0)
(9, 742, 0)
(9, 746, 0)
(9, 750, 0)
(9, 754, 0)
(9, 758, 0)
(9, 762, 0)
(9, 766, 0)
(9, 770, 0)
(9, 774, 0)
(9, 778, 0)
(9, 782, 0)
(9, 786, 0)
(9, 790, 0)
(9, 794, 0)
(9, 798, 0)
(9, 802, 0)
(9, 806, 0)
(9, 810, 0)
(9, 814, 0)
(9, 818, 0)
(9, 822, 0)
(9, 826, 0)
(9, 830, 0)
(9, 834, 0)
(9, 838, 0)
(9, 842, 0)
(9, 846, 0)
(9, 850, 0)
(9, 854, 0)
(9, 858, 0)
(9, 862, 0)
(9, 866, 0)
(9, 870, 0)
(9, 874, 0)
(9, 878, 0)
(9, 

(17, 206, 0)
(17, 210, 0)
(17, 214, 0)
(17, 218, 0)
(17, 222, 0)
(17, 226, 0)
(17, 230, 0)
(17, 234, 0)
(17, 238, 0)
(17, 242, 0)
(17, 246, 0)
(17, 250, 0)
(17, 254, 0)
(17, 258, 0)
(17, 262, 0)
(17, 266, 0)
(17, 270, 0)
(17, 274, 0)
(17, 278, 0)
(17, 282, 0)
(17, 286, 0)
(17, 290, 0)
(17, 294, 0)
(17, 298, 0)
(17, 302, 0)
(17, 306, 0)
(17, 310, 0)
(17, 314, 0)
(17, 318, 0)
(17, 322, 0)
(17, 326, 0)
(17, 330, 0)
(17, 334, 0)
(17, 338, 0)
(17, 342, 0)
(17, 346, 0)
(17, 350, 0)
(17, 354, 0)
(17, 358, 0)
(17, 362, 0)
(17, 366, 0)
(17, 370, 0)
(17, 374, 0)
(17, 378, 0)
(17, 382, 0)
(17, 386, 0)
(17, 390, 0)
(17, 394, 0)
(17, 398, 0)
(17, 402, 0)
(17, 406, 0)
(17, 410, 0)
(17, 414, 0)
(17, 418, 0)
(17, 422, 0)
(17, 426, 0)
(17, 430, 0)
(17, 434, 0)
(17, 438, 0)
(17, 442, 0)
(17, 446, 0)
(17, 450, 0)
(17, 454, 0)
(17, 458, 0)
(17, 462, 0)
(17, 466, 0)
(17, 470, 0)
(17, 474, 0)
(17, 478, 0)
(17, 482, 0)
(17, 486, 0)
(17, 490, 0)
(17, 494, 0)
(17, 498, 0)
(17, 502, 0)
(17, 506, 0)
(17, 510, 0)

(20, 998, 0)
(20, 1002, 0)
(24, 6, 0)
(24, 10, 0)
(24, 14, 0)
(24, 18, 0)
(24, 22, 0)
(24, 26, 0)
(24, 30, 0)
(24, 34, 0)
(24, 38, 0)
(24, 42, 0)
(24, 46, 0)
(24, 50, 0)
(24, 54, 0)
(24, 58, 0)
(24, 62, 0)
(24, 66, 0)
(24, 70, 0)
(24, 74, 0)
(24, 78, 0)
(24, 82, 0)
(24, 86, 0)
(24, 90, 0)
(24, 94, 0)
(24, 98, 0)
(24, 102, 0)
(24, 106, 0)
(24, 110, 0)
(24, 114, 0)
(24, 118, 0)
(24, 122, 0)
(24, 126, 0)
(24, 130, 0)
(24, 134, 0)
(24, 138, 0)
(24, 142, 0)
(24, 146, 0)
(24, 150, 0)
(24, 154, 0)
(24, 158, 0)
(24, 162, 0)
(24, 166, 0)
(24, 170, 0)
(24, 174, 0)
(24, 178, 0)
(24, 182, 0)
(24, 186, 0)
(24, 190, 0)
(24, 194, 0)
(24, 198, 0)
(24, 202, 0)
(24, 206, 0)
(24, 210, 0)
(24, 214, 0)
(24, 218, 0)
(24, 222, 0)
(24, 226, 0)
(24, 230, 0)
(24, 234, 0)
(24, 238, 0)
(24, 242, 0)
(24, 246, 0)
(24, 250, 0)
(24, 254, 0)
(24, 258, 0)
(24, 262, 0)
(24, 266, 0)
(24, 270, 0)
(24, 274, 0)
(24, 278, 0)
(24, 282, 0)
(24, 286, 0)
(24, 290, 0)
(24, 294, 0)
(24, 298, 0)
(24, 302, 0)
(24, 306, 0)
(24, 310, 

(34, 550, 0)
(34, 554, 0)
(34, 558, 0)
(34, 562, 0)
(34, 566, 0)
(34, 570, 0)
(34, 574, 0)
(34, 578, 0)
(34, 582, 0)
(34, 586, 0)
(34, 590, 0)
(34, 594, 0)
(34, 598, 0)
(34, 602, 0)
(34, 606, 0)
(34, 610, 0)
(34, 614, 0)
(34, 618, 0)
(34, 622, 0)
(34, 626, 0)
(34, 630, 0)
(34, 634, 0)
(34, 638, 0)
(34, 642, 0)
(34, 646, 0)
(34, 650, 0)
(34, 654, 0)
(34, 658, 0)
(34, 662, 0)
(34, 666, 0)
(34, 670, 0)
(34, 674, 0)
(34, 678, 0)
(34, 682, 0)
(34, 686, 0)
(34, 690, 0)
(34, 694, 0)
(34, 698, 0)
(34, 702, 0)
(34, 706, 0)
(34, 710, 0)
(34, 714, 0)
(34, 718, 0)
(34, 722, 0)
(34, 726, 0)
(34, 730, 0)
(34, 734, 0)
(34, 738, 0)
(34, 742, 0)
(34, 746, 0)
(34, 750, 0)
(34, 754, 0)
(34, 758, 0)
(34, 762, 0)
(34, 766, 0)
(34, 770, 0)
(34, 774, 0)
(34, 778, 0)
(34, 782, 0)
(34, 786, 0)
(34, 790, 0)
(34, 794, 0)
(34, 798, 0)
(34, 802, 0)
(34, 806, 0)
(34, 810, 0)
(34, 814, 0)
(34, 818, 0)
(34, 822, 0)
(34, 826, 0)
(34, 830, 0)
(34, 834, 0)
(34, 838, 0)
(34, 842, 0)
(34, 846, 0)
(34, 850, 0)
(34, 854, 0)

(48, 106, 0)
(48, 110, 0)
(48, 114, 0)
(48, 118, 0)
(48, 122, 0)
(48, 126, 0)
(48, 130, 0)
(48, 134, 0)
(48, 138, 0)
(48, 142, 0)
(48, 146, 0)
(48, 150, 0)
(48, 154, 0)
(48, 158, 0)
(48, 162, 0)
(48, 166, 0)
(48, 170, 0)
(48, 174, 0)
(48, 178, 0)
(48, 182, 0)
(48, 186, 0)
(48, 190, 0)
(48, 194, 0)
(48, 198, 0)
(48, 202, 0)
(48, 206, 0)
(48, 210, 0)
(48, 214, 0)
(48, 218, 0)
(48, 222, 0)
(48, 226, 0)
(48, 230, 0)
(48, 234, 0)
(48, 238, 0)
(48, 242, 0)
(48, 246, 0)
(48, 250, 0)
(48, 254, 0)
(48, 258, 0)
(48, 262, 0)
(48, 266, 0)
(48, 270, 0)
(48, 274, 0)
(48, 278, 0)
(48, 282, 0)
(48, 286, 0)
(48, 290, 0)
(48, 294, 0)
(48, 298, 0)
(48, 302, 0)
(48, 306, 0)
(48, 310, 0)
(48, 314, 0)
(48, 318, 0)
(48, 322, 0)
(48, 326, 0)
(48, 330, 0)
(48, 334, 0)
(48, 338, 0)
(48, 342, 0)
(48, 346, 0)
(48, 350, 0)
(48, 354, 0)
(48, 358, 0)
(48, 362, 0)
(48, 366, 0)
(48, 370, 0)
(48, 374, 0)
(48, 378, 0)
(48, 382, 0)
(48, 386, 0)
(48, 390, 0)
(48, 394, 0)
(48, 398, 0)
(48, 402, 0)
(48, 406, 0)
(48, 410, 0)

(66, 690, 0)
(66, 694, 0)
(66, 698, 0)
(66, 702, 0)
(66, 706, 0)
(66, 710, 0)
(66, 714, 0)
(66, 718, 0)
(66, 722, 0)
(66, 726, 0)
(66, 730, 0)
(66, 734, 0)
(66, 738, 0)
(66, 742, 0)
(66, 746, 0)
(66, 750, 0)
(66, 754, 0)
(66, 758, 0)
(66, 762, 0)
(66, 766, 0)
(66, 770, 0)
(66, 774, 0)
(66, 778, 0)
(66, 782, 0)
(66, 786, 0)
(66, 790, 0)
(66, 794, 0)
(66, 798, 0)
(66, 802, 0)
(66, 806, 0)
(66, 810, 0)
(66, 814, 0)
(66, 818, 0)
(66, 822, 0)
(66, 826, 0)
(66, 830, 0)
(66, 834, 0)
(66, 838, 0)
(66, 842, 0)
(66, 846, 0)
(66, 850, 0)
(66, 854, 0)
(66, 858, 0)
(66, 862, 0)
(66, 866, 0)
(66, 870, 0)
(66, 874, 0)
(66, 878, 0)
(66, 882, 0)
(66, 886, 0)
(66, 890, 0)
(66, 894, 0)
(66, 898, 0)
(66, 902, 0)
(66, 906, 0)
(66, 910, 0)
(66, 914, 0)
(66, 918, 0)
(66, 922, 0)
(66, 926, 0)
(66, 930, 0)
(66, 934, 0)
(66, 938, 0)
(66, 942, 0)
(66, 946, 0)
(66, 950, 0)
(66, 954, 0)
(66, 958, 0)
(66, 962, 0)
(66, 966, 0)
(66, 970, 0)
(66, 974, 0)
(66, 978, 0)
(66, 982, 0)
(66, 986, 0)
(66, 990, 0)
(66, 994, 0)

(80, 294, 0)
(80, 298, 0)
(80, 302, 0)
(80, 306, 0)
(80, 310, 0)
(80, 314, 0)
(80, 318, 0)
(80, 322, 0)
(80, 326, 0)
(80, 330, 0)
(80, 334, 0)
(80, 338, 0)
(80, 342, 0)
(80, 346, 0)
(80, 350, 0)
(80, 354, 0)
(80, 358, 0)
(80, 362, 0)
(80, 366, 0)
(80, 370, 0)
(80, 374, 0)
(80, 378, 0)
(80, 382, 0)
(80, 386, 0)
(80, 390, 0)
(80, 394, 0)
(80, 398, 0)
(80, 402, 0)
(80, 406, 0)
(80, 410, 0)
(80, 414, 0)
(80, 418, 0)
(80, 422, 0)
(80, 426, 0)
(80, 430, 0)
(80, 434, 0)
(80, 438, 0)
(80, 442, 0)
(80, 446, 0)
(80, 450, 0)
(80, 454, 0)
(80, 458, 0)
(80, 462, 0)
(80, 466, 0)
(80, 470, 0)
(80, 474, 0)
(80, 478, 0)
(80, 482, 0)
(80, 486, 0)
(80, 490, 0)
(80, 494, 0)
(80, 498, 0)
(80, 502, 0)
(80, 506, 0)
(80, 510, 0)
(80, 514, 0)
(80, 518, 0)
(80, 522, 0)
(80, 526, 0)
(80, 530, 0)
(80, 534, 0)
(80, 538, 0)
(80, 542, 0)
(80, 546, 0)
(80, 550, 0)
(80, 554, 0)
(80, 558, 0)
(80, 562, 0)
(80, 566, 0)
(80, 570, 0)
(80, 574, 0)
(80, 578, 0)
(80, 582, 0)
(80, 586, 0)
(80, 590, 0)
(80, 594, 0)
(80, 598, 0)